## Intro to TensorFlow

Note I am reading the book as my guide -- https://www.amazon.com/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1491962291 - quite a good book so far.

The goal here will be to explore building similar tools as I've done before:  
- Basic linear regression solver
- Basic logistic regression solver
- Gradient Descent solver

Then to move onto the next major area: Neural Networks



TensorFlow requires you to wrap your variables and methods into their own custom structures.  A few caveats:  
- TF wants stuff in a different row/matrix orientation than SciKit.  Everyone wants it different which is annoying (but just requires a transpose so not a huge deal)
- TF seems hard to debug since you wrap stuff and evaluate later in a graph
- TF graph gui stuff looks slick
- Its not so intuitive as a Py declarative programmer

Code that does a basic BGD Logistic Regression Solver -- note TF isn't installed then it won't work   
(Just pip install tensorflow)

In [3]:
import tensorflow as tf

tf.reset_default_graph()
n_epochs = 400
learning_rate = 0.01

xs = np.array([[10,1],[11,2],[1,6]])   # dummy sample 
ys = np.array([[1],[1],[0]])           # dummy results

X = tf.constant(xs, dtype=tf.float32, name='X')   # wrap in TF vanilla consts
y = tf.constant(ys, dtype=tf.float32, name='y')   

theta = tf.Variable(tf.constant([[0.1],[0.1]]), name='theta')  # TF "variable"
y_pred = tf.sigmoid(tf.matmul(X, theta, name='predictions'))  # wrap in TF sigmoid

with tf.name_scope("loss"):                # named scope (for graph imagery gropuing)
    error = y_pred - y                     # error used in next scope
    ll = tf.reduce_mean(tf.losses.log_loss(y,y_pred), name='log_loss')  # std log_loss function, not used?

with tf.name_scope("gradients"):
    gradients = 2.0/len(ys) * tf.matmul(tf.transpose(X), error)         # std partial deriv/gradient formula 
    training_op = tf.assign(theta, theta - learning_rate * gradients)   # "training_op" is called later

init = tf.global_variables_initializer()   # boilerplate init

with tf.Session() as sess:                 # this is where the TF stuff actually runs
    sess.run(init)
    for epoch in range(n_epochs):          # GD loop
        sess.run(training_op)              # each loop calls "training_op" again which assigns the theta
    best_theta = theta.eval()              # fetch theta array
    print(best_theta)


ModuleNotFoundError: No module named 'tensorflow'

-----


I was curious when GPU are faster.  I found a few things initially.

- CPU without MMX is slow (no vector operations on matrix and BGD)
- CPU w/ MMX is faster (should measure if I can disable MMX somehow)
- GPU is not always faster (on basic MNIST NN image example it was ~8% faster)

To setup GPU:
- Suggest setup a virtualenv of conda env, so you keep the env clean if you want to compare GPU vs non-GPU
- Install TensorFlow (tensorflow-gpu which includes tensorflow - 1.8 in my case)
- Install CUDA drivers (9.0 in my case, its compiled for only this version in Windows)
- Install Cuda NN DLL (7.1 - just copy DLL after installed to somewhere in path)

Note my initial installs were screwed up.  I reinstalled tensorflow after installing CUDA and now its working.



